In [52]:
from dataclasses import dataclass, field
import numpy as np

import scipy.integrate as sp
from scipy.spatial.transform import Rotation as Rot

In [53]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [54]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)
    
def rot3_to_scipy_rotation(rot3: Rot3) -> Rot:
    quat = rot3.as_quat()  # Use as_quat for scipy Rotation
    return Rot.from_quat([quat[0], quat[1], quat[2], quat[3]])

In [55]:
from dataclasses import dataclass, field 
@dataclass
class Params:
    #utility, quadrotor parameters
    mass: float = 1.352  # kg
    inertia: np.ndarray = field(default_factory=lambda: np.array([[19.8e-3, 0, 0], [0, 10.02e-3, 0], [0, 0, 18.6e-3]]))
    rotor_diameter: float = 10 * 0.0254  # meters, 10in
    static_thrust_coefficient: float = 0.14553
    static_torque_coefficient: float = 0.01047
    arm_length: float = 0.3814 / 2.0  # meters
    g: float = 9.80665  # m/s^2
    rho: float = 1.225  # kg/m^3
    
    def __init__(self):
        # Proportional and derivative gains for our controller
        self.K_p = np.diag([1.0, 1.0, 1.0])  # 3x3 matrix for position control
        self.K_d = np.diag([0.1, 0.1, 0.1])  # 3x3 matrix for velocity control
        
        # Mass of the quadrotor
        self.mass = 1.0  # in kg
        
        # Gravitational acceleration
        self.g = 9.81  # m/s^2
        
    def rotor_model(self, static_coefficient: float) -> float:
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

    @property
    def mixing_matrix(self):
        ratio = self.k_torque / self.k_thrust
        return np.array([
            [1, 1, 1, 1],
            [0, self.arm_length, 0, -self.arm_length],
            [-self.arm_length, 0, self.arm_length, 0],
            [ratio, -ratio, ratio, -ratio]
        ])
params = Params()

### Writing a controller for our drone
In this notebook, we'll start writing a feedback controller for our model! Again, I will be leaving you guys alone a little bit more.

#### Recommended steps
- Copy your 3D quadrotor model from the previous lecture
- Work your way down in the `Controller` class
    - I implemented a basic structure, but feel free to diverge from this if you like!
    - Write the inverse mixing matrix formulation to from **u** to rotor rates
    - As discussed, the first week I expect you to just set up control in *z* to warm up!

### The dynamics

In [56]:
class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self, params: Params) -> None:
        super().__init__()
        self.params = params  # Initialize params

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray, params: Params) -> np.ndarray:
        return np.sign(rotor_rates) * rotor_rates**2 * params.k_thrust
        
    def step(self, t: float, input: np.ndarray) -> QuadrotorState:
        # Access rotor rates directly from the input
        forces = FullQuadrotorDynamics.rotor_thrust_model(input, self.params)  # Use input directly
        
        u = (self.params.mixing_matrix @ forces.reshape(-1, 1)).reshape(-1)
        u1, u2 = u[0], u[1:]
    
    def state_derivative(t: float, state: np.ndarray) -> np.ndarray:
        return FullQuadrotorDynamics.state_derivative(
            QuadrotorState.from_state_vector(state),
            u1,
            u2,
        ).to_state_vector()

        state_vector = self.state.to_state_vector()
        sol = sp.solve_ivp(state_derivative, (0, self.dt), state_vector)
        self.state = QuadrotorState.from_state_vector(sol.y[:, -1])
        return self.state

    @staticmethod
    def state_derivative(state: QuadrotorState, u1: float, u2: np.ndarray) -> QuadrotorState:
        p = Params()
        accel = state.orientation * np.array([0, 0, -p.g]).T / p.mass + np.array([0, 0, -p.g])
        omega = state.angular_velocity
        angular_accel = np.linalg.inv(p.inertia) @ (u2 - np.cross(omega, p.inertia @ omega))

        dstate = QuadrotorState(
            position=state.velocity, 
            velocity=accel, 
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )
        return dstate

### The controller

In [57]:
# Define the controller parameters class
@dataclass
class ControllerParams:
#The same as above but for the controller gains*
#NOTE(roy) diag Just means we get a matrix with these values on the diagonal,
# so these are 3x3 matrices (print them if you're unsure!)
#We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 1000]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([10.5, 0.5, 10]))
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([150, 50, 20]))
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600


controller_p = ControllerParams()

In [64]:

class Controller:
    def __init__(self, params):
        self.params = params  # Store Params with K_p, K_d, mass, and gravity

    def step(self, t, trajectory, state):
        # Get desired and actual states
        desired_position = trajectory.position
        desired_velocity = np.zeros(3)
        
        # Assume the desired velocity is zero for simplicity
        print(state)
        actual_position = state.position
        actual_velocity = state.velocity
        # Calculate position and velocity errors
        p_error = desired_position - actual_position
        d_error = desired_velocity - actual_velocity

        # Control law (PD control)
        commanded_acceleration = self.params.K_p @ p_error + self.params.K_d @ d_error
        
        # Calculate thrust (u1) to balance the quadrotor using z-axis acceleration
        u1 = self.params.mass * (self.params.g + commanded_acceleration[2])

        # For now, we keep u2 (torques) as zeros since we are not controlling rotation
        u2 = np.zeros(3)

        # Return the control inputs: thrust (u1) and torques (u2)
        return np.array([u1, *u2])

In [65]:
@dataclass
class QuadrotorState:
    """Class to represent the state of the quadrotor."""
    position: np.ndarray
    velocity: np.ndarray
    orientation: Rot
    angular_velocity: np.ndarray

    def to_euler(self, seq: str):
        return self.orientation.as_euler(seq)

    def to_state_vector(self):
        return np.hstack([self.position, self.velocity, self.orientation.as_rotvec(), self.angular_velocity])

    @staticmethod
    def from_state_vector(state_vec: np.ndarray):
        position = state_vec[:3]
        velocity = state_vec[3:6]
        orientation = Rot.from_rotvec(state_vec[6:9])
        angular_velocity = state_vec[9:12]
        return QuadrotorState(position, velocity, orientation, angular_velocity)

class Quadrotor:
    
    def __init__(self, params: Params, initial_state: QuadrotorState):
        self.params = params
        self.state = initial_state
        self.dt = 0.01

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray, params: Params) -> np.ndarray:
        return np.sign(rotor_rates) * rotor_rates**2 * params.k_thrust

    def state_derivative(self, t: float, state_vector: np.ndarray, u: np.ndarray) -> np.ndarray:
        state = QuadrotorState.from_state_vector(state_vector)
        u1, u2 = u[0], u[1:]

        # Compute acceleration
        accel = np.array([0, 0, u1]) / self.params.mass + np.array([0, 0, -self.params.g])
        omega = state.angular_velocity
        angular_accel = np.linalg.inv(self.params.inertia) @ (u2 - np.cross(omega, self.params.inertia @ omega))

        # Derivatives of state variables
        dstate = np.hstack([
            state.velocity,  # d(position)/dt = velocity
            accel,           # d(velocity)/dt = acceleration
            omega,           # d(orientation)/dt = angular velocity
            angular_accel    # d(angular velocity)/dt = angular acceleration
        ])

        return dstate

    def step(self, t: float, u: np.ndarray):
        state_vector = self.state.to_state_vector()
        sol = sp.solve_ivp(self.state_derivative, [0, self.dt], state_vector, args=(u,))
        self.state = QuadrotorState.from_state_vector(sol.y[:, -1])
        return self.state

In [66]:
#ratio = 50 / (2*np.sqrt(p.mass*100))
#ratio

### The trajectory
Because we're now finally doing closed-loop control, we also need to set desired states in the trajectory!
I've set up something simple, please mess around with it!

In [67]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x = 0 if t < 1.0 or t > 5.0 else 1.0
        y = 0 if t < 2.0 or t > 5.0 else 1.0
        position = np.array([x, y, 0])
        yaw_rate = 0.5
        yaw = yaw_rate * t  # Need to the yaw calculation on Sun
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)

### Running it!

In [68]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [69]:
def run_and_render(initial_state: QuadrotorState, t_total: float = 2.0):
    # Initializing the simulator with the correct controller and parameters
    params = Params()  # Make sure to define Params
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(params),  # Pass params to FullQuadrotorDynamics
        controller=Controller(params),  # Pass params to the Controller
        trajectory=JumpTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )

    # Run the simulator
    output = sim.simulate()
    plot = animate_k3d(output)
    plot.display()
    plot.start_auto_play()

# Define initial state
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

# Run the simulation
run_and_render(initial_state, 5.0)

QuadrotorState(position=array([0., 0., 0.]), velocity=array([0, 0, 0]), orientation=<Rot3 [0.0, 0.0, 0.0, 1.0]>, angular_velocity=array([0., 0., 0.]))
None


AttributeError: 'NoneType' object has no attribute 'position'

In [ ]:
# NOTE(roy) play with this variable and see what happens!
params = Params()

initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

# Modify the `run_and_render` function to include params
def run_and_render(initial_state: QuadrotorState, t_total: float = 2.0):
    # Initialize the simulator with the correct controller and parameters
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(params),  # Pass the params to the Controller
        trajectory=JumpTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )

    # Run the simulator
    output = sim.simulate()
    return output

# Run the simulation
run_and_render(initial_state, 5.0)

### Looking at the controller behavior
If all is well, the quadrotor should have a stable hover and after 1 second jump up 1 meter (with the `JumpTrajectory`)

Try some experiments:
- What happens if you increase/descrease `K_p`?
- What happens if you increase/decrease `K_d`?
- What happens if you use a slightly different mass in the controller (so the controller and dynamics have a different estimate for the quadrotor mass)? How do you think you could solve the behavior you see?

### Commit your updated notebook to your fork!